In [1]:
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch

In [2]:
pickle_in = open('input.pkl', 'rb')
pickledata = pickle.load(pickle_in)

In [5]:
pickle_in2 = open('label.pkl', 'rb')
labeldata = pickle.load(pickle_in2)

In [29]:
pickledata[:,0,0, 0] # all the values for x accel


tensor([ -6.7445,  -2.7466,   0.2441,  ..., -53.5897, -54.6883, -55.3292])

In [38]:
# separate out a section of the data, first 500 data points at 1/125
sampledata = pickledata[0:500, 0:, 0, 0].numpy()
samplelabels = labeldata[0:500, 0:, 0, 0].numpy()  # predict the left foot

In [39]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(sampledata, samplelabels, test_size=0.2, random_state=42)

In [41]:
np.shape(x_train) # sanity check passed

(400, 6)

In [45]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

reg = LinearRegression().fit(x_train, y_train)
linregpred = reg.predict(x_test)
mean_squared_error(y_test, linregpred)

0.13435695

In [50]:
from sklearn import linear_model 
ridgereg = linear_model.Ridge()
ridgereg.fit(x_train, y_train)
ridgeregpred = ridgereg.predict(x_test)
mean_squared_error(y_test, ridgeregpred)

0.1432707

In [52]:
lassoreg = linear_model.Lasso(alpha=0.1)
lassoreg.fit(x_train, y_train)
lassoregpred = lassoreg.predict(x_test)
mean_squared_error(y_test, lassoregpred)

0.14908259

In [60]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline
poly = PolynomialFeatures(degree=2)
x_train_new = poly.fit_transform(x_train)
x_test_new = poly.fit_transform(x_test)

polymodel = Pipeline([('poly', PolynomialFeatures(degree=2)),('linear', LinearRegression())])
polymodel = polymodel.fit(x_train_new, y_train)
polymodelpred = polymodel.predict(x_test_new)
mean_squared_error(y_test,polymodelpred)

0.04599841